<a href="https://colab.research.google.com/github/CaQtiml/Data-Structure-and-Graph-Algorithm/blob/master/DecisionTree_fromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset

In [114]:
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [115]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [116]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [117]:
#######
# Demo:
print(unique_vals(training_data, 0))
print(unique_vals(training_data, 1))
#######

{'Yellow', 'Red', 'Green'}
{1, 3}


In [118]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
      label = row[-1]
      if label not in counts:
        counts[label]=0
      counts[label]+=1
    return counts

In [119]:
#######
# Demo:
x = class_counts(training_data)
print(type(x))
#######

<class 'dict'>


In [120]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [121]:
class Question:
  """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
  def __init__(self,column,value):
    self.column = column
    self.value = value
  
  def match(self,example): # example be like ['Yellow', 3, 'Apple']
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value
  def __repr__(self):
      # This is just a helper method to print
      # the question in a readable format.
      condition = "=="
      if is_numeric(self.value):
          condition = ">="
      return "Is %s %s %s?" % (
          header[self.column], condition, str(self.value))

In [122]:
#######
# Demo:
# Let's write a question for a numeric attribute
print(Question(0, "Yellow"))
print(Question(1, 3))

Is color == Yellow?
Is diameter >= 3?


In [123]:
q = Question(0, 'Green')
example = training_data[0]
print(example)
q.match(example)

['Green', 3, 'Apple']


True

In [124]:
def partition(rows, question):
  """Partitions a dataset.

  For each row in the dataset, check if it matches the question. If
  so, add it to 'true rows', otherwise, add it to 'false rows'.

  rows - training set
  question - Question(0, 'Red')

  OUTPUT : true_rows ([['Red', 1, 'Grape'], ['Red', 1, 'Grape']]) , false_rows ([['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']])
  """
  true_rows = []
  false_rows = []

  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows,false_rows



In [125]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [126]:
def gini(rows):
  counts = class_counts(rows)
  impurity = 1
  for lbl in counts:
    prob_of_lbl = counts[lbl]/float(len(rows))
    impurity -= prob_of_lbl**2
  return impurity

In [127]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [128]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [129]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [130]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [131]:
## another implementation of info_gain by receiving all instead of gini(all) (but same algorithm)
def info_gain_2(left,right,all):
  p = float(len(left)) / (len(left) + len(right))
  return gini(all) - p*gini(left) - (1-p)*gini(right)

In [132]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain_2(true_rows, false_rows, training_data)

0.1399999999999999

In [133]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [134]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(info_gain(true_rows, false_rows, current_uncertainty))
true_rows, false_rows = partition(training_data, Question(0,'Red'))
print(info_gain(true_rows, false_rows, current_uncertainty))

0.1399999999999999
0.37333333333333324


We choose the way that returns high info_gain because its impurity after splitting is low.

It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
Why? Look at the different splits that result, and see which one looks more 'unmixed' to you.

In [135]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))
print(true_rows)
print(false_rows)
# at least it can distinguish "Grape".

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [136]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# We've isolated one apple in the true rows.
print(true_rows)
print(false_rows)

[['Green', 3, 'Apple']]
[['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [137]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):
      values = set([row[col] for row in rows])
      for val in values:
        question = Question(col,val)
        true_rows,false_rows = partition(rows,question)
        if len(true_rows)==0 or len(false_rows)==0 :
          continue
        gain = info_gain(true_rows,false_rows,current_uncertainty)
        # print(f"{val}:{gain}")
        if gain >= best_gain:
          best_gain,best_question = gain,question
    return best_gain,best_question

In [138]:

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

In [139]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [140]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [141]:
def build_tree(rows):
  best_gain,best_question = find_best_split(rows)
  if best_gain == 0 :
    return Leaf(rows)
  true_rows,false_rows = partition(rows,best_question)
  true_node = build_tree(true_rows)
  false_node = build_tree(false_rows)
  return Decision_Node(best_question,true_node,false_node)

In [142]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [143]:
my_tree = build_tree(training_data)

In [144]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [145]:
def classify(row,node):
  if isinstance(node,Leaf):
    return node.predictions
  if node.question.match(row):
    return classify(row,node.true_branch)
  else:
    return classify(row,node.false_branch)

In [146]:
classify(["Green",3], my_tree)

{'Apple': 1}

In [147]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [148]:
print_leaf(classify(["Green",3], my_tree))
print_leaf(classify(["Yellow",3], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [149]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [150]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
